In [1]:
import qokit
import numpy as np
from itertools import combinations

# Simulation of QAOA applied to a general problem

In this tutorial, we show to get QAOA objective for a problem that does not have a directly implemented high-level API like LABS or MaxCut. In particular, we will consider the Sherrington-Kirkpatrick (SK) model given by the following objective:
$$
G(z) = -\sum_{1\leq i <j\leq N}J^{(N)}_{ij}z_iz_j
$$
where $J^{(N)}_{ij} \sim \mathcal{N}(\mu(N), \sigma^2)$.

To simulate the QAOA on SK model, we need to generate a problem representation. The format used by QOKit is a of tuples, where each tuple defines a summand and contains indices of the Pauli Zs in the product along with the coefficient. For example, if terms = [(0.5, (0,1)), (-1, (0,1,2,3)), (1,(1,2))], the Hamiltonian is
$$
0.5z_0z_1 - z_0z_1z_2z_3 + z_1z_2
$$

In [2]:
N = 4
np.random.seed(10)
terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]

## Create simulator

`simclass` is a class that you should use to create the simulator.
There are multiple simulators in `qokit.fur` module.
You can choose to use `gpu` or `python` simulator and the following notebook should run without error

In [3]:
simclass = qokit.fur.choose_simulator(name='auto')
sim = simclass(N, terms=terms)

LazyModule: mpi4py.MPI is missing.


In [4]:
### Get precomputed diagonal cost vector and cache if desired

In [5]:
sim.get_cost_diagonal()

array([ 0.39433175, -0.60859862, -3.49474551,  0.82867014,  0.42071262,
        2.27889815, -3.50190003,  3.68263151,  3.68263151, -3.50190003,
        2.27889815,  0.42071262,  0.82867014, -3.49474551, -0.60859862,
        0.39433175])

## Simulate QAOA for parameters

Simulator returns a `result` object which may be different depending on the type of simulator you use.
If you want to use it directly, you have to know exactly which simulator you are using

In [6]:
p = 3
gamma, beta = np.random.rand(2, 3)
_result = sim.simulate_qaoa(gamma, beta) # Result depends on the type of simulator. 

## Get simulation results

### Get statevector

In [7]:
sv = sim.get_statevector(_result)
sv

array([0.13720186+0.14502526j, 0.09431783+0.12211294j,
       0.04665999-0.1383308j , 0.16514588+0.16112639j,
       0.15497791+0.17550594j, 0.26061719+0.187216j  ,
       0.03410968-0.17167419j, 0.36219128+0.20537478j,
       0.36219128+0.20537478j, 0.03410968-0.17167419j,
       0.26061719+0.187216j  , 0.15497791+0.17550594j,
       0.16514588+0.16112639j, 0.04665999-0.1383308j ,
       0.09431783+0.12211294j, 0.13720186+0.14502526j])

### Get probabilities

The simulator will calculate probabilities for you. This may be done in-place, overwriting the data of statevector to avoid allocation of a separate array

In [8]:
probs = sim.get_probabilities(_result)
probs.sum()

1.0000000000000004

This will overwrite the `_result` if using GPU simulation. Subsequent calls to `get_statevector` return invalid data.

In [9]:
probs = sim.get_probabilities(_result, preserve_state=False)
sv2 = sim.get_statevector(_result)
print("Using numpy") if np.allclose(sv, sv2) else print("Yohoo, I'm using a memory-economic simulator!")

Using numpy


### Get expectation value

For numpy version, the simulator effectively does `sv.dot(costs)`, where costs is the precomputed diagonal.

You may specify your own observable vector which will be used instead of the diagonal hamiltonian.

In [10]:
e = sim.get_expectation(_result)
costs_inv = 1/sim.get_cost_diagonal()
e_inv = sim.get_expectation(_result, costs=costs_inv)
print("Expectation of C:", e)
print("1/(Expectation of 1/C): ", 1/e_inv)

Expectation of C: 1.5194551853006746
1/(Expectation of 1/C):  1.4973912754030274


### Get overlap

Returns the overlap with ground states, which are states corresponding to minimum of C. 
You may specify the corresponding costs as well. Additionally, prodive `indices` if you want to get probability for particular state.

In [11]:
overlap = sim.get_overlap(_result)
print("Ground state overlap:", overlap)
# Below we test that for positive-valued cost function, the maximum can be achieved 
# by either inverting the values, or negating the values.
costs_abs = np.abs(sim.get_cost_diagonal())
print("Overlap with ground state for absolute cost:", sim.get_overlap(_result, costs=costs_abs))
overlap_inv = sim.get_overlap(_result, costs=1/costs_abs)
print("Overlap with highest state (inverted costs):", overlap_inv)
overlap_neg = sim.get_overlap(_result, costs=-costs_abs)
print("Overlap with highest state (negative):", overlap_neg)
assert overlap_inv == overlap_neg, "You may have values of mixed sign in your cost."

Ground state overlap: 0.06127099876467231
Overlap with ground state for absolute cost: 0.0797133524682188
Overlap with highest state (inverted costs): 0.346722642613863
Overlap with highest state (negative): 0.346722642613863


Overlap may be calculated in-place as well, which may make subsequent calls like `sim.get_expectation()` return incorrect values.
The default behavior is to copy, to provide consistent between simulator types.

If you only need one value for each `result`, you can pass `preserve_state=False` to reduce the memory usage.

In [12]:
# specify state indices
overlap_03 = sim.get_overlap(_result, indices=[0, 3])
probs = sim.get_probabilities(_result)
assert overlap_03 == probs[[0, 3]].sum(), "This is a bug, please report it"

---